In [3]:
import pandas as pd
import torch
import datasets
from datasets import Dataset, DatasetDict, load_dataset
from trl import KTOConfig, KTOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [23]:
cache_dir = "../assets/pretrained-models"
model_path = "meta-llama/Llama-3.2-1B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    cache_dir=cache_dir,
    use_safetensors=True,
    do_sample=False
)

tokenizer = AutoTokenizer.from_pretrained(
    model_path, cache_dir=cache_dir, use_safetensors=True, padding_side="left"
)

tokenizer.pad_token = tokenizer.eos_token

In [26]:
Claimbuster = pd.read_json('../data/Claimbuster/train.json')
dataset = Claimbuster.copy(deep=True)
dataset['messages'] = dataset['text'].apply(lambda x: [{"role": "user", "content": f"Is the following sentence a factual claim? {x}"}, {"role": "system", "content": "You are a yes/no answering bot designed to tell whether or not a statement is a factual claim that could be checked by fact-checkers. Only respond to questions with Yes or No"}, {"role": "assistant", "content": ""}])
dataset['label'] = dataset['label'].apply(lambda x: "Yes" if x == 1 else "No")
dataset = dataset.filter(items=['text','messages','label'])
dataset.head()

,text,messages,label
0,The 9/11 Commission put out a report that said...,"[{'role': 'user', 'content': 'Is the following...",Yes
1,"Our Government has, in the last year or so, ta...","[{'role': 'user', 'content': 'Is the following...",Yes
2,"It's now 121/4, and I predict it'll be coming ...","[{'role': 'user', 'content': 'Is the following...",Yes
3,"They've appropriated about $200 million, but o...","[{'role': 'user', 'content': 'Is the following...",Yes
4,"One seventh the total economy, 17 new taxes, p...","[{'role': 'user', 'content': 'Is the following...",Yes


In [30]:
train_dataset = dataset.sample(n=100, random_state=42).to_dict(orient='records')

results_df = pd.DataFrame(columns=["text", "label", "pred", "eval"])
for i, train_instance in enumerate(train_dataset):
    print(f"DEBUG::i::{i} / {len(train_dataset)}")
    print("DEBUG::text::", train_instance['text'])
    chat_template_input_ids = tokenizer.apply_chat_template(train_instance['messages'], tokenize=True, continue_final_message=True, add_generation_prompt=False, return_tensors="pt")
    chat_template_input_ids = chat_template_input_ids[0, :-1].reshape(1,-1)

    with torch.no_grad():
        logits = model(
            chat_template_input_ids,
            use_cache=True
        )["logits"]
    next_token = torch.argmax(logits[0,-1]).item()
    next_token_text = tokenizer.convert_ids_to_tokens(next_token)
    print("DEBUG::label::", train_instance['label'])
    print("DEBUG::response::", next_token_text)

    r = {
        "text": train_instance['text'],
        "label": train_instance['label'],
        "pred": next_token_text,
        "eval": 1 if (next_token_text in train_instance['label']) else 0
    }
    results_df.loc[len(results_df)] = r

results_df

DEBUG::text:: And I don't want historians, ten years from now, to say, these were the years when the tide ran out for the United States.
DEBUG::response:: Yes
DEBUG::text:: And we're struggling with that today.
DEBUG::response:: No
DEBUG::text:: I know they're popular, but I'm against them.
DEBUG::response:: No
DEBUG::text:: When -- when he was asked would class size...
DEBUG::response:: No
DEBUG::text:: China, is a huge country, broken into many provinces.
DEBUG::response:: No
DEBUG::text:: You even brought some right-wing congressman into the White House to plot how to attack me for going to Russia in 1969-70, when over 50,000 other Americans did.
DEBUG::response:: No
DEBUG::text:: We have to remember, when we talk a tax cut, we're only talking about reducing a tax increase, because this Administration has left us with a built-in tax increase that will amount to $86 billion next year.
DEBUG::response:: No
DEBUG::text:: That's what brings jobs in.
DEBUG::response:: No
DEBUG::text:: An

,text,label,pred,eval
0,"And I don't want historians, ten years from no...",No,Yes,0
1,And we're struggling with that today.,No,No,1
2,"I know they're popular, but I'm against them.",No,No,1
3,When -- when he was asked would class size...,No,No,1
4,"China, is a huge country, broken into many pro...",No,No,1
...,...,...,...,...
95,Parents are teaching their children right from...,No,Yes,0
96,They don't name him the most liberal in the Un...,Yes,No,0
97,They didn't get the tanks from this administra...,Yes,No,0
98,Small businesses are not going to have a mandate.,No,Yes,0


In [31]:
results_df['eval'].mean()

np.float64(0.46)